In [1]:
import pandas as pd
import warnings
import requests, zipfile, io, os, re
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
warnings.filterwarnings('ignore')

In [2]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [3]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [4]:
download_and_extract(healthy_url, healthy_path)

In [5]:
download_and_extract(parkinson_url, parkinson_path)

In [6]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'
healthy = []
parkinson = []

In [7]:
for filename in os.listdir(healthy_signals):
  if re.search('sigSp', filename) is not None:
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#')
    healthy.append(df)

for filename in os.listdir(parkinson_signals):
 if re.search('sigSp', filename) is not None:
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#')
    parkinson.append(df)

In [8]:
def preprocess(X, y):
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
  X_valid = scaler.transform(X_valid.reshape(-1, X_valid.shape[-1])).reshape(X_valid.shape)
  return X_train, X_valid, y_train, y_valid

In [9]:
def create_dataset(healthy, parkinson, seq):
  X = []
  y = []

  # healthy signals
  for df in healthy:
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(0)

  # parkinson signals
  for df in parkinson:
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(1)

  return np.array(X), np.array(y)


In [10]:
def train_model(X_train, X_valid, y_train, y_valid):
  model = keras.models.Sequential([
  keras.layers.GRU(128, return_sequences=True, input_shape=(X_train.shape[1], 6)),
  keras.layers.GRU(128),
  keras.layers.Dense(1, activation="sigmoid")
  ])

  model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  history = model.fit(X_train, y_train, epochs = 50, batch_size = 64, validation_data=(X_valid, y_valid))
  return history



In [ ]:
sequence_list = [100, 200, 300, 400,500,600,700]
stats = []


for sequence in sequence_list:
  X, y = create_dataset(healthy, parkinson, sequence)
  X_train, X_valid, y_train, y_valid = preprocess(X, y)
  history = train_model(X_train, X_valid, y_train, y_valid)
  stats.append(history)
  print(sequence)



Epoch 1/50
587/587 [==============================] - 29s 49ms/step - loss: 0.4761 - accuracy: 0.7833 - val_loss: 0.4280 - val_accuracy: 0.8122
Epoch 2/50
587/587 [==============================] - 27s 45ms/step - loss: 0.3800 - accuracy: 0.8302 - val_loss: 0.3727 - val_accuracy: 0.8286
Epoch 3/50
587/587 [==============================] - 26s 44ms/step - loss: 0.3257 - accuracy: 0.8570 - val_loss: 0.3130 - val_accuracy: 0.8641
Epoch 4/50
587/587 [==============================] - 26s 44ms/step - loss: 0.2811 - accuracy: 0.8788 - val_loss: 0.2686 - val_accuracy: 0.8885
Epoch 5/50
587/587 [==============================] - 26s 44ms/step - loss: 0.2361 - accuracy: 0.9022 - val_loss: 0.2589 - val_accuracy: 0.8925
Epoch 6/50
587/587 [==============================] - 25s 43ms/step - loss: 0.1992 - accuracy: 0.9180 - val_loss: 0.2018 - val_accuracy: 0.9189
Epoch 7/50
587/587 [==============================] - 25s 43ms/step - loss: 0.1746 - accuracy: 0.9292 - val_loss: 0.1826 - val_accuracy:

In [ ]:
# val accuracy
for i, sequence in enumerate(sequence_list):
  plt.plot(stats[i].history['val_accuracy'], label=f'{sequence}')

plt.legend()
plt.show()

In [ ]:
# train accuracy
for i, sequence in enumerate(sequence_list):
  plt.plot(stats[i].history['accuracy'], label=f'{sequence}')

plt.legend()
plt.show()

In [ ]:
# val loss
for i, sequence in enumerate(sequence_list):
  plt.plot(stats[i].history['val_loss'], label=f'{sequence}')

plt.legend()
plt.show()

In [ ]:
# loss
for i, sequence in enumerate(sequence_list):
  plt.plot(stats[i].history['loss'], label=f'{sequence}')

plt.legend()
plt.show()

In [ ]:
sequence_list = [20, 40, 80, 100]
stats = []


for sequence in sequence_list:
  X, y = create_dataset(healthy, parkinson, sequence)
  X_train, X_valid, y_train, y_valid = preprocess(X, y)
  history = train_model(X_train, X_valid, y_train, y_valid)
  stats.append(history)
  print(sequence)

In [ ]:
# val accuracy
for i, sequence in enumerate(sequence_list):
  plt.plot(stats[i].history['val_accuracy'], label=f'{sequence}')

plt.legend()
plt.show()